In [1]:
source("helper.R")

max_expid<-get_max("Log")+1
exp_name<-"sensitivityOfStartPoint"
log_dir<-glue("Log/{max_expid}.{exp_name}")
logger=get_logger(log_dir, log_name = "log", debug = TRUE)
futile.logger::flog.info("start experiment", name = "log")

## get logger

use_saudi<-FALSE
seed_random<-123
n=30

futile.logger::flog.info(glue("use_saudi: {use_saudi}"), name = "log")

set.seed(seed_random)

futile.logger::flog.info(glue("use seed: {seed_random}"), name = "log")

futile.logger::flog.info(glue("number of experiments: {n}"), name = "log")

if(!use_saudi){

    data_ori <- as.data.frame(read_csv("data/IFclose.csv"))
    logpt<-log(data_ori$close)
    logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
    date <- as.Date(as.character(data_ori$Date[1:(length(data_ori$Date)-1)]), format = "%Y%m%d")
    date_timeindex<-as.numeric(as.POSIXct(date)) 
    date_timeindex <- (date_timeindex - min(date_timeindex))/86400 ## 1 day = 86400 seconds
    mesh <- fm_mesh_1d(date_timeindex)
    data = data.frame(logreturn = logreturn, times = date_timeindex)
    data <- rbind(data, data.frame(logreturn = NA, times = max(date_timeindex):(max(date_timeindex)+31)))
    mesh <- fm_mesh_1d(data$times)
}else{
    data_ori <- as.data.frame(read_csv("data/1180_vol.csv"))
    logpt<-log(data_ori$Close)
    logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)] ## 1 day = 86400 seconds
    date <- data_ori$Date[1:(length(data_ori$Date)-1)]
    date_timeindex<-as.numeric(as.POSIXct(date)) 
    date_timeindex <- (date_timeindex - min(date_timeindex))/86400
    mesh <- fm_mesh_1d(date_timeindex)
    data = data.frame(logreturn = logreturn, times = date_timeindex)
    data <- rbind(data, data.frame(logreturn = NA, times = max(date_timeindex):(max(date_timeindex)+31)))
    mesh <- fm_mesh_1d(data$times)
}



Loading required package: Matrix



Warning message in file(con, "r"):
“URL 'https://inla.r-inla-download.org/VERSIONS': Timeout of 2 seconds was reached”
This is INLA_25.06.07 built 2025-06-11 18:54:45 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


This is rSPDE 2.5.1
- See https://davidbolin.github.io/rSPDE for vignettes and manuals.



INFO [2025-08-27 21:55:32] start experiment
INFO [2025-08-27 21:55:32] use_saudi: FALSE
INFO [2025-08-27 21:55:32] use seed: 123
INFO [2025-08-27 21:55:32] number of experiments: 30


Rows: 1595 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): ins_id
dbl (2): Date, close

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:


params <- data.frame(
  rspde.order = sample(2:6, n, replace = TRUE),
  prior.range.nominal = runif(n, 5, 300),
  prior.std.dev.nominal = runif(n, 0.3, 1.7)
)




# 

for(model_family in c("stochvol","stochvol.t","stochvol.nig")){
  results_list <- list()
  mlik_vec <- numeric(n)


  futile.logger::flog.info(glue("Start fitting models:{model_family}"), name = "log")
  for (i in 1:n) {
    futile.logger::flog.info(glue("🔁 Fitting model {i} of {n} ..."), name = "log")
    
    # 提取参数
    rspde.order <- params$rspde.order[i]
    prior.range.nominal <- params$prior.range.nominal[i]
    prior.std.dev.nominal <- params$prior.std.dev.nominal[i]
    
    # 构建模型
    OU_model <- rspde.matern(
      mesh = mesh,
      parameterization = "matern",
      rspde.order = rspde.order,
      prior.range.nominal = prior.range.nominal,
      prior.std.dev.nominal = prior.std.dev.nominal
    )
    
    formula_OU <- logreturn ~ -1 + Intercept(1)+ field(times, model = OU_model)
    
    fit <- tryCatch(
      bru(
        formula_OU,
        family = model_family,
        data = data
      ),
      error = function(e) {
        futile.logger::flog.warn(glue("⚠️ Fit failed at i = {i} : {e$message}"), name = "log")
        return(NULL)
      }
    )
    
    # 如果成功拟合
    if (!is.null(fit)) {
      result_fit <- rspde.result(fit, "field", OU_model, parameterization = "matern")
      mlik_vec[i] <- fit$mlik[2]  # marginal log-likelihood
      results_list[[i]] <- list(
        param = params[i, ],
        mlik = fit$mlik[2],
        result = result_fit
      )
    } else {
      mlik_vec[i] <- NA
      results_list[[i]] <- list(param = params[i, ], mlik = NA, result = NULL)
    }
  }
  sorted_idx <- order(mlik_vec, decreasing = TRUE)


  results_list_sorted <- results_list[sorted_idx]


  table_out <- data.frame(
    param = character(),
    std.dev = numeric(),
    range = numeric(),
    nu = numeric(),
    likelihood = numeric(),
    stringsAsFactors = FALSE
  )
  results_list_sorted <- results_list[sorted_idx]
  for (i in seq_along(results_list_sorted)) {
    res <- results_list_sorted[[i]]
    
    if (is.null(res$result)) next 
    
    param_str <- with(res$param, glue::glue("({rspde.order}/{round(prior.range.nominal, 4)}/{round(prior.std.dev.nominal, 4)})"))
    
    summary_res <- summary(res$result)
    
    table_out[i, ] <- list(
      param = param_str,
      std.dev = round(summary_res$mean[1], 3),
      range   = round(summary_res$mean[2], 3),
      nu      = round(summary_res$mean[3], 3),
      likelihood = round(res$mlik, 3)
    )
  }

  table_out <- na.omit(table_out)


  table_string<-knitr::kable(
    table_out,
    format = "latex",
    booktabs = TRUE,
    linesep = ""
  )

  file_name <- glue("{log_dir}/model_with_{model_family}.tex")
  writeLines(table_string,  file_name)

}

INFO [2025-08-27 21:55:32] Start fitting models:stochvol
INFO [2025-08-27 21:55:32] 🔁 Fitting model 1 of 30 ...
INFO [2025-08-27 21:55:50] 🔁 Fitting model 2 of 30 ...
INFO [2025-08-27 21:56:07] 🔁 Fitting model 3 of 30 ...
INFO [2025-08-27 21:56:20] 🔁 Fitting model 4 of 30 ...
INFO [2025-08-27 21:56:32] 🔁 Fitting model 5 of 30 ...
INFO [2025-08-27 21:56:45] 🔁 Fitting model 6 of 30 ...
INFO [2025-08-27 21:57:23] 🔁 Fitting model 7 of 30 ...
INFO [2025-08-27 21:57:38] 🔁 Fitting model 8 of 30 ...
INFO [2025-08-27 21:57:48] 🔁 Fitting model 9 of 30 ...
INFO [2025-08-27 21:58:01] 🔁 Fitting model 10 of 30 ...
INFO [2025-08-27 21:58:15] 🔁 Fitting model 11 of 30 ...
INFO [2025-08-27 21:58:44] 🔁 Fitting model 12 of 30 ...
INFO [2025-08-27 21:59:03] 🔁 Fitting model 13 of 30 ...
INFO [2025-08-27 21:59:27] 🔁 Fitting model 14 of 30 ...
INFO [2025-08-27 21:59:34] 🔁 Fitting model 15 of 30 ...
INFO [2025-08-27 21:59:50] 🔁 Fitting model 16 of 30 ...
INFO [2025-08-27 22:00:02] 🔁 Fitting model 17 of 30 ...


Warning message in rspde.result(fit, "field", OU_model, parameterization = "matern"):
“the mean or mode of nu is very close to nu.upper.bound, please consider increasing nu.upper.bound, and refitting the model.”


INFO [2025-08-27 22:03:50] 🔁 Fitting model 2 of 30 ...


Warning message in rspde.result(fit, "field", OU_model, parameterization = "matern"):
“the mean or mode of nu is very close to nu.upper.bound, please consider increasing nu.upper.bound, and refitting the model.”


INFO [2025-08-27 22:04:13] 🔁 Fitting model 3 of 30 ...


...

Warning message in rspde.result(fit, "field", OU_model, parameterization = "matern"):
“the mean or mode of nu is very close to nu.upper.bound, please consider increasing nu.upper.bound, and refitting the model.”


INFO [2025-08-27 22:24:17] 🔁 Fitting model 29 of 30 ...


Warning message in rspde.result(fit, "field", OU_model, parameterization = "matern"):
“the mean or mode of nu is very close to nu.upper.bound, please consider increasing nu.upper.bound, and refitting the model.”


INFO [2025-08-27 22:24:30] 🔁 Fitting model 30 of 30 ...


Warning message in rspde.result(fit, "field", OU_model, parameterization = "matern"):
“the mean or mode of nu is very close to nu.upper.bound, please consider increasing nu.upper.bound, and refitting the model.”
